# Computação Gráfica 2025/2026
## Aula TP09 - Introdução ao Ray Casting e Ray Tracing

### Um tutorial gráfico por André Falcão (2025)

Nesta aula vamos aprender como podemos representar imagens mais realistas. Começaremos com o Ray Casting e depois iremos para o Ray Tracing. Estes algoritmos serão descritos em detalhe, e incluem toda a matéria discutida até agora

1. Tracing de raios e lidar com intersecções
2. Algoritmo de Ray Casting
3. Algoritmo de Ray Tracing

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


## 1. Funções básicas

Antes de implementar qualquer dos algoritmos, precisamos de resolver duas questões básicas. a) como podemos descobrir os elementos que aparecem quando um raio é emitido; e b) dado um raio, quais as intersecções que temos na cena. Como faremos uma abordagem bottom-up criaremos as funções antes de serem chamadas no código

### 1.1. Lidar com intersecções

A primeira coisa que necessitaremos é de uma função para detectar intersecções. Se temos um raio com um ponto de origem e um vector, a pergunta que queremos responder é, onde é que esse vector interesecciona esse objecto

Como a nossa cena serão só esferas, a função de intersecção pode ser muito simplificada, dado que, se sabemos o  centro  e o raio só temos que verificar se o raio chega à esfera, e se sim, qual a distância a que está o ponto de intersecção  

Para  
* Raio: $r(t) = \mathbf{ro} + t.\mathbf{rd}$, onde **ro** é a origem e **rd** é a direcção (normalizada).
* Esfera: centro **c** e raio **r**.
* Procuramos o menor $t > 0$ tal que o ponto do raio está na superfície da esfera.


A condição de intersecção é:
$$ \|\mathbf{ro} + t\,\mathbf{rd} - \mathbf{c}\|^2 = r^2. $$

Definindo $\mathbf{oc} = \mathbf{ro}-  \mathbf{c}$:

$$ \|\mathbf{oc} + t\,\mathbf{rd}\|^2 = r^2 \;\Longrightarrow\; (\mathbf{rd}\cdot\mathbf{rd})\,t^2 + 2(\mathbf{oc}\cdot\mathbf{rd})\,t + (\mathbf{oc} \cdot \mathbf{oc} - r^2) = 0. $$

Se $\mathbf{rd}$ está normalizado, então $\mathbf{rd} \cdot \mathbf{rd}=1$ e ficamos com a forma:

$$ t^2 + 2b\,t + c = 0, $$

onde
$$ b = \mathbf{oc}\!\cdot\!\mathbf{rd}, \qquad c = \mathbf{oc}\!\cdot\!\mathbf{oc} - r^2. $$

As raízes desta forma simplificada da equação quadrática são:

$$  t = -b \pm \sqrt{\,b^2 - c\,} $$


E escolhemos o valor menor.

O código em baixo é uma implementação directa desta análise:


In [ ]:
def intersect_sphere(ro, rd, c, r):
    oc = ro - c
    b = np.dot(oc, rd)
    cval = np.dot(oc, oc) - r*r
    disc = b*b - cval
    if disc < 0: return np.inf
    s = np.sqrt(disc)
    t1, t2 = -b - s, -b + s
    
    #vamos escolher apenas distâncias positivas e suficientemente distantes do observador (daí o threshold de 0.0001)
    # eo valor é o menor possível
    t = t1 if (t1 > 1e-4) else (t2 if t2 > 1e-4 else np.inf)
    return t


### 1.2. Emissão de Raios

A seguir necessitaremos de uma função em que, dado um ponto (ro) e um vector (rd), no espaço da cena devolve o ponto que encontrou, usando naturalmente a função de intersecção anterior

O retorno é:
* `hit` - Ponto em que o raio atingiu o objecto
* `idx` - identificador do objecto (forma ou triángulo)
* `t` - componente de `rd` que aplicado a `ro` dá o `hit`: $ hit=ro+t.rd $

In [ ]:
def trace(ro, rd):
    t_min, hit, idx = np.inf, None, -1
    for i, (c, r, col, shin, refl) in enumerate(spheres):
        t = intersect_sphere(ro, rd, c, r)
        if t < t_min:
            t_min, hit, idx = t, ro + t*rd, i
    return hit, idx, t_min



## 2. Algoritmo de Ray Casting

O Ray Casting tem uma forma muito simples de ser processado. É assim

1. Varre os vários pixeis definidos no viewport, sabendo as coordenadas (ro inicial) e orientação da câmara
2. Para cada pixel x,y, determina o seu vector (rd) criando um "raio" que percorre a cena, determinando o objecto que lhe está mais próximo
3. Para esse objecto, aplica-lhe a função de iluminação apropriada de acordo com as suas propriedades

Uma implementação directa destes princípios pode ser vista na função seguinte

In [ ]:
def shade_ray_casting(ro, rd):
    #Identificar em que é que se embate - se nada, é o background e é o fim
    hit, idx, t = trace(ro, rd)
    if hit is None:
        return bg

    #recolher a info do material do objecto atingido (ver as esferas já a seguir)
    c, r, base_col, shin, refl = spheres[idx]
    n = (hit - c) / r #calcular a normal
    n = n / np.linalg.norm(n)  # Ensure normalized
    
    # calcular a posição da luz relativamente ao ponto e distância
    to_light = light_pos - hit
    dist_l = np.linalg.norm(to_light)
    ldir = to_light / dist_l

    ambient = 0.1
    diff=max(0.0, np.dot(n, ldir))
    h = (ldir - rd) / np.linalg.norm(ldir - rd)
    spec=max(0.0, np.dot(n, h)) ** shin
    final_color = base_col * (ambient + 0.9*diff) + light_col * (0.3*spec)
    
    return np.clip(final_color, 0, 1)



### 2.2. Definir a cena

Vamos definir a cena apenas com 4 esferas. Uma delas de raio 1000 e que servirá como "chão"

Define-se ainda uma luz apenas com posição e cor

In [ ]:
#centro, raio, cor, especularidade
spheres = [
    (np.array([ 0.0, -0.3, 0.0]), 0.4, np.array([0.9, 0.2, 0.2]), 50, 0.3),  # ligeiramente reflectiva vermelha
    (np.array([ 1.1, -0.2, 0.3]), 0.3, np.array([0.2, 0.9, 0.2]), 50, 0.0),  # Verde sem reflexão
    (np.array([ 0.6,  0.3, 0.5]), 0.25, np.array([0.8, 0.8, 0.9]), 200, 0.8), # Esfera Metálica
    (np.array([ 0.0, -1000.5, -2.0]), 1000.0, np.array([0.9, 0.9, 0.95]), 8, 0.1) # chão ligeiramente reflexivo
]

#Luz
light_pos = np.array([2.1, .5, 0.5])
light_col = np.array([1.0, 1.0, 1.0])

#cor de fundo <- azul céu
bg = np.array([0.7, 0.85, 1.0])


Os parâmetros da cena são definidos em baixo com uma imagem de 640x480,  fov vertical de 60 graus e posição da câmara em $[0, 0.1, 3.0])$

In [ ]:

W, H = 640, 480
fov = 60 * np.pi/180
cam_pos = np.array([0, 0.1, 3.0])


### 2.3. Definição da câmara

O setup da câmara é uma inversão do LookAt (que já conhecemos) retornando os vectores forward, right e up para processamento na parte de produção de raios

**Derivação 1: $V^{-1}$ a partir de $V$**

A **view matrix** leva coordenadas do **mundo** para **câmara**:
$$
V=\begin{bmatrix}
R^\top & -R^\top \mathbf{eye}\\
\mathbf{0}^\top & 1
\end{bmatrix},
\qquad
\begin{bmatrix}\mathbf{x}\\1\end{bmatrix}
=
V\begin{bmatrix}\mathbf{p}\\1\end{bmatrix}
=
\begin{bmatrix}
R^\top(\mathbf{p}-\mathbf{eye})\\[2pt]
1
\end{bmatrix}.
$$

Logo, em coordenadas 3D não-homogéneas, que transforma qualquer ponto em coordenadas do mundo em coordenadas da câmara (composição de uma rotação e de uma translação)
$$
\mathbf{x}=R^\top(\mathbf{p}-\mathbf{eye}).
$$

Isolando $p$:
$$
\mathbf{p}-\mathbf{eye}=R\,\mathbf{x}
\;\;\Longrightarrow\;\;
\mathbf{p}=R\,\mathbf{x}+\mathbf{eye}.
$$

Em coordenadas homogéneas, isto corresponde a aplicar a inversa:
$$
\begin{bmatrix}\mathbf{p}\\1\end{bmatrix}
=
\underbrace{\begin{bmatrix}
R & \mathbf{eye}\\
\mathbf{0}^\top & 1
\end{bmatrix}}_{\displaystyle V^{-1}}
\begin{bmatrix}\mathbf{x}\\1\end{bmatrix}.
$$

Esta matriz $V^{-1}$ é muito semelhante à matriz V e pode ser produzida com um processo idêntico, recebendo os mesmos parâmetros de entrada. Na nossa função, por razões práticas, vamos apenas fazer sair os vectores `forward`, `right` e `up`


#### Exercício
* Ver na TP correspondente como foi calculada a Matriz LookAt ($V$) e comparar com esta e com a forma como é aplicada



In [ ]:

def setup_camera(eye, target, up):
    #simplificação do LookAt
    forward = target - eye
    forward = forward / np.linalg.norm(forward)
    
    # Vector right
    right = np.cross(forward, up)
    right = right / np.linalg.norm(right)
    
    # garantir vectores ortogonais
    up = np.cross(right, forward)
    up = up / np.linalg.norm(up)
    
    return forward, right, up


### 2.4. Derivação de `px` e `py` para Ray Casting (pinhole invertido)

Agora que sabemos como transformar qualquer ponto de coordenadas da câmara para coordenadas do mundo, vamos obter esses pontos, varrendo o ecrã. Par isso vamos precisar das seguintes fases:
1. Obter a base das coordenadas da câmara
2. Inverter a função perspectiva usando o plano de imagem e o Campo de visão (FOV)
3. Calcular as coordenadas normalizadas do pixel
4. Calcular cada ponto correspondente no plano da imagem


**1. Definição da base ortonormada da câmara**
Seja uma base ortonormal da câmara obtida pela função anterior:
- **forward**: $ \mathbf{f} $
- **right**: $ \mathbf{r} $
- **up**: $ \mathbf{u} $

Assume-se $ \{\mathbf{r},\mathbf{u},\mathbf{f}\} $ ortonormais e com orientação de mão direita.

**2. Inverter a função perspectiva**

Coloca-se o **plano da imagem** a uma distância focal $f$ do centro de câmara, ao longo de $ \mathbf{f} $.
Para **FOV vertical** (fovy), a meia-altura do plano é:
$$
h = f\,\tan\!\left(\frac{\mathrm{fov}}{2}\right),
$$
e a **meia-largura** é
$$
w = h \cdot \mathrm{aspect}, \quad \text{com } \mathrm{aspect} = \frac{W}{H}.
$$

NOTA: se fizermos $f=1$ simplificamos as fórmulas, como veremos a seguir

**3. Calcular as coordenadas normalizadas do pixel (NDC)**

Para o pixel **inteiro** $(x,y)$, com $x\in[0,W-1]$ e $y\in[0,H-1]$, usa-se o **centro do pixel** com $+0.5$ e mapeia-se para $(u,v)\in[-1,1]\times[-1,1]$:

O eixo $x$ (esquerda → direita) fica assim:
$$ u = \left(\frac{x+0.5}{W}\right)\cdot 2 - 1. $$

O eixo $y$ (topo → fundo) é ligeiramente diferente. Como a imagem tem $y$ a crescer para baixo, mas queremos $+v$ a apontar para **cima**, inverte-se:
$$ v = 1 - \left(\frac{y+0.5}{H}\right)\cdot 2. $$

**4.Projecção do ponto no plano da imagem**

Isto é o que queremos obter, que é, como cada pixel projectado entre $[-1,1]$, se pode converter para coordenadas físicas no plano da imagem, que com $f=1$:
 fica:

$$ x_{\text{img}} =  u \cdot w = u \cdot \mathrm{aspect}\cdot \tan\!\left(\frac{\mathrm{fov}}{2}\right), \quad
y_{\text{img}} = v \cdot h = v \cdot \tan\!\left(\frac{\mathrm{fov}}{2}\right). $$


Que correspondem aos pixeis x e y em **cordenadas de câmara.** A aplicação dos vectores da função de projecção (provenientes do setup_camera) projecta essas coordenadas em coordenadas do mundo, e assim conseguimos criar o vector `rd` para definir o raio inicial juntamente com as coordenadas da câmara

### 2.5. A função principal de Ray Casting

A função de ray casting é a função principal e, depois de definir os parâmetros da câmara, varre cada um dos pixeis, invertendo a função de perspectiva para calcular a posição dos pixeis e depois definir o vector (ray direction - `rd`) com a direcção. O ponto de origem (ray origin `ro`) é a posição da câmara

Esta função salva uma imagem em PNG e devolve essa mesma imagem para ser mostrada no ecrã

In [ ]:
def ray_cast(fname="ray_cast.png"):
    cam_target = np.array([0, 0, 0])     # Point camera is looking at
    cam_up = np.array([0, 1, 0])         # Up direction
    aspect = W / H
    cam_forward, cam_right, cam_UP = setup_camera(cam_pos, cam_target, cam_up)
    img = np.zeros((H, W, 3), dtype=np.float32)
    for y in range(H):
        for x in range(W):
            #posição do Pixel (x,y)
            px = ((x + 0.5) / W - 0.5) * 2.0 * aspect * np.tan(fov/2)
            py = ((H - y - 0.5) / H - 0.5) * 2.0 * np.tan(fov/2)

            #agora projectar para coordenadas do mundo e normalizar
            rd = px * cam_right + py * cam_UP + cam_forward
            rd = rd / np.linalg.norm(rd)
            
            # e aqui está o procedimento final<- determinação da cor através do ray casting.
            color = shade_ray_casting(cam_pos, rd) 
            img[y, x] = color

    Image.fromarray((np.clip(img,0,1)*255).astype(np.uint8)).save(fname)
    print("Guardado:"+fname)
    return img


Podemos agora criar uma imagem

In [ ]:
img = ray_cast()
plt.imshow(img, clim=[0, 1])  
plt.axis('off') 
plt.show()


## 3. Ray tracing

O algoritmo de Ray-tracing é muito semelhante ao de Ray Casting envolvendo mais 2 componentes. Há uma 3a componente (transmissão em objectos transparentes) que não será tratada aqui.

Vamos aproveitar quase tudo o que temos do ray casting pois muitos dos pricípios são os mesmos, pelo que não necessitaremos de redifinir a detecção de colisões de raios nem de recriar a cena e as suas propriedades

1. Varre os vários pixeis definidos no viewport, sabendo as coordenadas (ro inicial) e orientação da câmara
2. Para cada pixel x,y, determina o seu vector (rd) criando um "raio" que percorre a cena, determinando o objecto que lhe está mais próximo
3. Para esse objecto e ponto de contacto cria um raio novo em direcção à fonte de iluminação e,
   1. caso não haja nada no caminho, aplica-lhe a função de iluminação apropriada de acordo com as suas propriedades
   2. Caso exista algum obstáculo, aplica-lhe a luz ambiente
4. Para esse mesmo ponto, caso ele tenha alguma reflectividade, vai ver a cor associada aos raios que podem ter atingido esse ponto, propagando recursivamente este algoritmo de acordo com a condição de paragem
5. [**Não vista nesta aula**] Caso o objecto tenha alguma transparência vai fazer um raio através das propriedades transmissivas do objecto e propagá-lo para a cena recursivamente de acordo com a condição de paragem

A função `shade_ray_tracing` inclui essas modificações, sendo no restante idêntica à função `shade_ray_casting` De notar que existe um parâmetro extra de profundidade (`depth`) que controla a profundidade de recursão do algoritmo e uma nova variável global de controlo que define esses limites

A função auxiliar  `reflect`,  também definida aqui, dá a direcção de reflexão dado um vector de raio e a normal do objecto embatido e será útil a seguir

In [ ]:

def reflect(rd, n):
    # Calcula a direcção de reflexão: rd - 2*(rd·n)*n
    return rd - 2 * np.dot(rd, n) * n


def shade_ray_tracing(ro, rd, depth=0):
    # Fim da recursão se atingimos o limite
    if depth > MAX_DEPTH:
        return bg * 0.1  #background atenuado

    #Identificar em que é que se embate - se nada, é o background e é o fim
    hit, idx, t = trace(ro, rd)
    if hit is None:
        return bg

    #recolher a info do material do objecto atingido
    c, r, base_col, shin, refl = spheres[idx]
    n = (hit - c) / r #calcular a normal
    n = n / np.linalg.norm(n)  # Ensure normalized
    
    # calcular a posição da luz relativamente ao ponto e distância
    to_light = light_pos - hit
    dist_l = np.linalg.norm(to_light)
    ldir = to_light / dist_l

    #e ver se nada está no caminho da(s) luz(es)
    eps = 1e-3
    shadow_hit, idx2, tshadow = trace(hit + n*eps, ldir)
    in_shadow = (shadow_hit is not None) and (tshadow < dist_l - 1e-3)

    #e define-se a cor local (Phong)
    ambient = 0.1
    if in_shadow:
        diff = 0.0 
        spec = 0.0
    else: 
        diff=max(0.0, np.dot(n, ldir))
        h = (ldir - rd) / np.linalg.norm(ldir - rd)
        spec=max(0.0, np.dot(n, h)) ** shin
    local_color = base_col * (ambient + 0.9*diff) + light_col * (0.3*spec)
    
    # Se o material é reflexivo, Faz-se a recursão
    if refl > 0:
        reflect_dir = reflect(rd, n)
        reflect_color = shade_ray_tracing(hit + n*eps, reflect_dir, depth + 1)
        # finalmente faz o Blending da cor local com a cor proveniente dos reflexos
        final_color = local_color * (1 - refl) + reflect_color * refl
    else:
        final_color = local_color
    
    return np.clip(final_color, 0, 1)



### 3.1. O ponto de entrada

Os princípios são os mesmos do código anterior, mas atenção ao parâmatro `MD` que define desde logo qual a profundidade máxima a que o algoritmo correrá

In [ ]:

MAX_DEPTH = 4  # Valor máximo da recursão

def ray_trace(fname="ray_trace.png", MD=0):
    global MAX_DEPTH
    MAX_DEPTH = MD
    cam_target = np.array([0, 0, 0])     # Point camera is looking at
    cam_up = np.array([0, 1, 0])         # Up direction
    aspect = W / H
    cam_forward, cam_right, cam_UP = setup_camera(cam_pos, cam_target, cam_up)
    img = np.zeros((H, W, 3), dtype=np.float32)
    for y in range(H):
        for x in range(W):
            #posição do Pixel (x,y)
            px = ((x + 0.5) / W - 0.5) * 2.0 * aspect * np.tan(fov/2)
            py = ((H - y - 0.5) / H - 0.5) * 2.0 * np.tan(fov/2)

            #definição do Raio de acordo com os parâmetros calculados
            rd = px * cam_right + py * cam_UP + cam_forward
            rd = rd / np.linalg.norm(rd)
            # e aqui está tudo<- determinação da cor através do ray tracing.
            color = shade_ray_tracing(cam_pos, rd) 
            img[y, x] = color

    Image.fromarray((np.clip(img,0,1)*255).astype(np.uint8)).save(fname)
    print("Guardado:"+fname)
    return img


In [ ]:
img = ray_trace(fname="ray_trace4.png", MD=4)
plt.imshow(img, clim=[0, 1])  
plt.axis('off')  
plt.show()



# Exercícios
1. Corra o ray tracing para `MAX_DEPTH` entre 0 e 4 ($[0,1,2,3,4]$) guardando em ficheiros distintos cada uma das imagens
   1. Compare os resultados entre si e com a imagem calculada com ray casting
   2. Compare os tempos de geração de cada imagem (incluindo o ray casting) e faça um gráfico (função `plt.plot(x,y)` do matplotlib) mostrando a evolução. Comente os resultados
   3. Discuta. Vale a pena (nesta cena) continuar a recursão?
2. Volte a executar `ray_trace(fname="ray_trace0.png", MD=0)`
   1. O que está a acontecer aqui?
   2. Há recursão?
   3. Se não há recursão porque é que não é idêntico ao ray casting e se acha que há recursão discuta como.
   4. Qual a grande vantagem e importância desta parâmetrização e o que consegue fazer aqui (e com MAX DEPTH>0) que não era até agora possível. Identifique no algoritmo e no código onde é que o efeito aqui produzido acontece e comente-o
3. Compare a função de iluminação dada aqui (e comum a ambos os algoritmos) e compare-a com outros métodos e parametrizações que já vimos nas aulas
   1. identifique as grandes diferenças
   2. Consegue melhorar o modelo de iluminação (modificando necessariamente a estrutura dos objectos) para que o modelo fique mais parecido com o que se viu nas aulas de Modelos de Iluminação Locais?
4. Adicione mais duas esferas à cena - uma mais pequena, amarela dourada à frente de todas, e outra maior, azul, atras de todas. Faça o rendering da cena e controle o tempo de execução. Discuta os resultados
5. Coloque a câmara em cima a olhar para baixo (ligeiramente oblíqua)
6. [PARA FAZER na AULA] ...
7. Discuta como poderia ser feito este processo para malhas poligonais. Quais os principais desafios?
   1. [PARA DISCUTIR NA AULA] ...
8. [OPCIONAL] O código em baixo consegue ler de um conjunto de PNGs gerados e criar uma animação.
   1. Crie um ciclo em que  se modifique um ou mais dos parâmetros da cena (por exemplo a posição da câmara, da luz, ou de uma das esferas, ou todas)
   2. para cada iteração do ciclo gere uma imagem PNG numa pasta separada, como acima, mas numerada. e.g. (`img000.png`, `img001.png`, ...)
   3. Crie o filme com o código abaixo (atenção à instalação do `imageio`)
   4. Compartilhe o resultado no forum de alunos! - Terá um bonus na nota o melhor video! 🙂


In [ ]:
# Atenção: Primeiro fazer: pip install imageio imageio[ffmpeg]
# o código pode dar um aviso devido às dimensões da janela, mas o video é gerado

import glob
import imageio
png_files=sorted(glob.glob("PNGS/*.png"))
fps=30
mp4_name = "ray-tracing-good.mp4"
print("A escrever vídeo MP4...")
with imageio.get_writer(mp4_name, fps=fps, codec="libx264", quality=8) as w:
    for f in png_files:
        w.append_data(imageio.imread(f))
print(f"Vídeo escrito: {mp4_name}")